# 下載課程所需檔案 (YOLOv7, Dataset)

In [ ]:
!wget https://github.com/TA-aiacademy/course_3.0/releases/download/CVCNN_Data/dog_cat_for_YOLO.zip
!unzip dog_cat_for_YOLO.zip
!wget https://github.com/TA-aiacademy/course_3.0/releases/download/CVCNN_Data/yolov7.zip
!unzip yolov7.zip

# 下載課程所需檔案(先訓練好的權重)

In [ ]:
!wget https://github.com/TA-aiacademy/course_3.0/releases/download/CVCNN_Data/pre_train.pt

# YOLOv7 預測教學

## 匯入所需要的套件

In [ ]:
import argparse
import time
from pathlib import Path

import cv2
import torch
import torch.backends.cudnn as cudnn
from numpy import random

from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import (check_img_size, check_requirements,
                           check_imshow, non_max_suppression, apply_classifier,
                           scale_coords, xyxy2xywh, strip_optimizer,
                           set_logging, increment_path)
from utils.plots import plot_one_box
from utils.torch_utils import select_device, load_classifier, time_synchronized

## 參數與初始值設定

In [ ]:
# 用 parser 來設定參數，參數不能重複設置
parser = argparse.ArgumentParser()

### 模型相關參數

In [ ]:
# 模型存放的路徑
parser.add_argument('--weights', nargs='+', type=str, default='yolov7.pt')
# 要預測的目標 file/folder, 0 for webcam
parser.add_argument('--source', type=str, default='inference/images')
# 預測的圖片大小
parser.add_argument('--img-size', type=int, default=640)
# 是物件的閾值調整
parser.add_argument('--conf-thres', type=float, default=0.25)
# 執行 NMS 的 IOU 閾值調整
parser.add_argument('--iou-thres', type=float, default=0.45)
# 設定 CPU/GPU 的裝置，正常電腦使用 GPU 會設 0
parser.add_argument('--device', default='cpu')

### 存檔相關參數

In [ ]:
# 是否把結果儲存成文字文件
parser.add_argument('--save-txt', action='store_true')
# 是否把信心分數存下來
parser.add_argument('--save-conf', action='store_true')
# 是否不儲存結果（圖片或影像的預測結果）
parser.add_argument('--nosave', action='store_true')
# 資料儲存位置，會儲存在 project/name 底下
parser.add_argument('--project', default='runs/detect')
# 資料儲存位置的子資料夾名稱，會儲存在 project/name 底下
parser.add_argument('--name', default='exp')
# 如果資料夾已存在，是否放進同個資料夾，或要新增資料夾來放
# 預設是 False，資料夾存在就會新增一個資料夾放結果
parser.add_argument('--exist-ok', action='store_true')

### NMS 相關參數

In [ ]:
# 可以指定只顯示特定的類別，--class 0, or --class 0 2 3
parser.add_argument('--classes', nargs='+', type=int)
# NMS 預設是相同類別會取一個框，可以設定同個位置只選一個框
# 跨類別的執行 NMS （目的是希望不要有很多框重複在同個位置，預設是 False）
parser.add_argument('--agnostic-nms', action='store_true')
# 預測時是否對目標做資料擴增
parser.add_argument('--augment', action='store_true')

In [ ]:
# 基本上使用預設即可，僅需調整權重位置即要預測的目標
opt = parser.parse_args(
    args=['--weights', './pre_train.pt',
          '--source', './datasets/pet/valid/images/Bengal_130.jpg',
          '--device', '0',
          '--nosave'])
print(opt)

In [ ]:
source, weights = opt.source, opt.weights
save_txt, imgsz = opt.save_txt, opt.img_size
save_img = not opt.nosave and not source.endswith('.txt')  # 是否要儲存預測結果

# 如果輸入是 webcam，會讀取 webcam 位置
webcam = source.isnumeric() or source.endswith('.txt') or \
    source.lower().startswith(('rtsp://', 'rtmp://', 'http://', 'https://'))

# 設定資料夾
# save_dir => 'runs/{project}/{name}'
save_dir = Path(increment_path(Path(opt.project) / opt.name,
                               exist_ok=opt.exist_ok))  
(save_dir / 'labels' if save_txt else save_dir).mkdir(
    parents=True, exist_ok=True)  # make dir

# 初始化設定
set_logging()
device = select_device(opt.device)
half = device.type != 'cpu'  # 如果是GPU的話，就使用半精度的方式做預測(速度會快一些)

In [ ]:
with torch.no_grad():

    # Load model
    model = attempt_load(weights, map_location=device)  # load FP32 model
    vid_path, vid_writer = None, None
    stride = int(model.stride.max())  # model stride

    # 設定 Dataloader
    if webcam:
        view_img = check_imshow()
        cudnn.benchmark = True  # 如果輸入的圖片大小都相同，設定這個可以增加推論速度，若輸入大小不同，反而會降低速度
        dataset = LoadStreams(source, img_size=imgsz, stride=stride)
    else:
        dataset = LoadImages(source, img_size=imgsz, stride=stride)

    if half:
        model.half()  # to FP16

    # Get names and colors
    names = model.module.names if hasattr(model, 'module') else model.names
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in names]

    # Run inference, 讓模型初始化一次
    if device.type != 'cpu':
        model(torch.zeros(1, 3, imgsz, imgsz).to(device).
              type_as(next(model.parameters())))  # run once

    t0 = time.time()
    for path, img, im0s, vid_cap in dataset:
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        # Inference
        t1 = time_synchronized()
        with torch.no_grad():
            pred = model(img, augment=opt.augment)[0]
        t2 = time_synchronized()

        # Apply NMS
        pred = non_max_suppression(pred, opt.conf_thres,
                                   opt.iou_thres,
                                   classes=opt.classes,
                                   agnostic=opt.agnostic_nms)
        t3 = time_synchronized()

        # Process detections
        for i, det in enumerate(pred):  # detections per image
            if webcam:  # batch_size >= 1
                p, s,  = path[i], '%g: ' % i
                im0, frame = im0s[i].copy(), dataset.count
            else:
                p, s,  = path, ''
                im0, frame = im0s, getattr(dataset, 'frame', 0)
            p = Path(p)  # to Path
            save_path = str(save_dir / p.name)  # img.jpg
            # img.txt
            txt_path = str(save_dir / 'labels' / p.stem) + \
                ('' if dataset.mode == 'image' else f'_{frame}')

            # normalization gain whwh
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]
            if len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(img.shape[2:],
                                          det[:, :4], im0.shape).round()

                # Print results
                for c in det[:, -1].unique():
                    # detections per class
                    n = (det[:, -1] == c).sum()
                    # add to string
                    s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "

                # Write results
                for *xyxy, conf, cls in reversed(det):
                    if save_txt:  # Write to file
                        # normalized xywh
                        xywh = (xyxy2xywh(torch.tensor(xyxy)
                                          .view(1, 4)) / gn).view(-1).tolist()
                        # label format
                        if opt.save_conf:
                            line = (cls, *xywh, conf)
                        else:
                            line = (cls, *xywh)
                        with open(txt_path + '.txt', 'a') as f:
                            f.write(('%g ' * len(line)).rstrip() % line + '\n')

                    if save_img:  # Add bbox to image
                        label = f'{names[int(cls)]} {conf:.2f}'
                        plot_one_box(xyxy, im0, label=label,
                                     color=colors[int(cls)],
                                     line_thickness=1)

            # Print time (inference + NMS)
            print(f'{s}Done. ({(1E3 * (t2 - t1)):.1f}',
                  f'ms) Inference, ({(1E3 * (t3 - t2)):.1f}ms) NMS')


            # 儲存預測完的結果
            if save_img:
                if dataset.mode == 'image':
                    cv2.imwrite(save_path, im0)
                    print("The image with the result is saved in:",
                          {save_path})
                else:  # 'video' or 'stream'
                    if vid_path != save_path:  # new video
                        vid_path = save_path
                        if isinstance(vid_writer, cv2.VideoWriter):
                            # release previous video writer
                            vid_writer.release()
                        if vid_cap:  # video
                            fps = vid_cap.get(cv2.CAP_PROP_FPS)
                            w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                            h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                        else:  # stream
                            fps, w, h = 30, im0.shape[1], im0.shape[0]
                            save_path += '.mp4'
                        vid_writer = cv2.VideoWriter(
                            save_path,
                            cv2.VideoWriter_fourcc(*'mp4v'),
                            fps, (w, h))
                    vid_writer.write(im0)

    if save_txt or save_img:
        s = f"\n{len(list(save_dir.glob('labels/*.txt')))} labels saved to {save_dir / 'labels'}" if save_txt else ''
        print(f"Results saved to {save_dir}{s}")

    print(f'Done. ({time.time() - t0:.3f}s)')

In [ ]:
!python detect.py --weights pre_train.pt --source ./datasets/pet/valid/images/Bengal_130.jpg